#### TODO
* Precision-Recall http://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html
* Use Git meta data as feature

#### Columns
0. id
1. repository_id
2. blamed_commit_id
3. type
4. sha
5. url
6. author_email
7. author_name
8. author_when
9. committer_email
10. committer_name
11. committer_when
12. additions
13. deletions
14. total_changes
15. past_changes
16. future_changes
17. past_different_authors
18. future_different_authors
19. author_contributions_percent
20. message
21. patch
22. hunk_count
23. cve
24. files_changed
25. patch_keywords

In [1]:
import psycopg2
conn = psycopg2.connect("dbname=postgres host=localhost port=55432 user=postgres")
cur = conn.cursor()
cur.execute("SELECT type, COUNT(id) FROM export.commits GROUP BY type")
cur.fetchall()


[('blamed_commit', 714L), ('fixing_commit', 1137L), ('other_commit', 349558L)]

In [11]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import cross_validation
import numpy as np
import scipy as sp
import psycopg2
import enum

class Colum(enum.IntEnum):
    id = 1
    type = 3
    patch = 21
    
class Data:
    def __init__(self, config):
        self.cur = psycopg2.connect(config).cursor()
    def fetchAll(self, sql):
        self.cur.execute(sql)
        return np.array(self.cur.fetchall())

def tfidf(text):
    # @TODO preprocessor: log number
    vectorizer = TfidfVectorizer(min_df=1)
    vectorized = vectorizer.fit_transform(text)
#     print vectorizer.get_feature_names()
    return vectorized.toarray()

# Fetch original data
data = Data("dbname=postgres host=localhost port=55432 user=postgres")
vcc = data.fetchAll("SELECT * FROM export.commits WHERE type  = 'blamed_commit' ORDER BY RANDOM() LIMIT 50")
ucc = data.fetchAll("SELECT * FROM export.commits WHERE type != 'blamed_commit' ORDER BY RANDOM() LIMIT 1000")

# Concat onece and shuffle
sample = np.concatenate([vcc, ucc])
np.random.shuffle(sample)
patches = sample[:, Colum.patch]
labels = sample[:, Colum.type]

x = tfidf(patches)
y = is_vcc = (labels == 'blamed_commit')
vcc_paches = patches[is_vcc]
ucc_paches = patches[~is_vcc]

clf = LinearSVC(C=1.0)
scores = cross_validation.cross_val_score(clf, x, y, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


Accuracy: 0.95 (+/- 0.00)


In [ ]:
# Precision-Recall
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.cross_validation import train_test_split
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import label_binarize
from sklearn import cross_validation
import numpy as np
import scipy as sp
import psycopg2
import enum

class Colum(enum.IntEnum):
    id = 1
    type = 3
    patch = 21
    
class Data:
    def __init__(self, config):
        self.cur = psycopg2.connect(config).cursor()
    def fetchAll(self, sql):
        self.cur.execute(sql)
        return np.array(self.cur.fetchall())

def tfidf(text):
    vectorizer = TfidfVectorizer(min_df=1)
    vectorized = vectorizer.fit_transform(text)
    return vectorized.toarray()

precision = dict()
recall = dict()
average_precision = dict()

# Fetch original data
data = Data("dbname=postgres host=localhost port=55432 user=postgres")
vcc = data.fetchAll("SELECT * FROM export.commits WHERE type  = 'blamed_commit' ORDER BY RANDOM() LIMIT 5")
ucc = data.fetchAll("SELECT * FROM export.commits WHERE type != 'blamed_commit' ORDER BY RANDOM() LIMIT 100")

# Concat onece and shuffle
sample = np.concatenate([vcc, ucc])
np.random.shuffle(sample)
patches = sample[:, Colum.patch]
labels = sample[:, Colum.type]

X = tfidf(patches)
y = is_vcc = (labels == 'blamed_commit')

# Split into training and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.5, random_state=0)

# Run classifier
classifier = LinearSVC(C=1.0)
y_score = classifier.fit(X_train, y_train).decision_function(X_test)

# Compute Precision-Recall and plot curve
precision[0], recall[0], _ = precision_recall_curve(y_test, y_score)
average_precision[0] = average_precision_score(y_test, y_score)

print precision, recall, average_precision

# Plot Precision-Recall curve
plt.clf()
plt.plot(recall[0], precision[0], label='Precision-Recall curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('Precision-Recall example: AUC={0:0.2f}'.format(average_precision[0]))
plt.legend(loc="lower left")
plt.show()

# # Plot Precision-Recall curve for each class
# plt.clf()
# plt.plot(recall[0], precision[0], label='Precision-recall curve of class {0} (area = {1:0.2f})'
#          ''.format(i, average_precision[0]))

# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('Recall')
# plt.ylabel('Precision')
# plt.title('Extension of Precision-Recall curve to multi-class')
# plt.legend(loc="lower right")
# plt.show()